In [34]:
import glob
import pandas as pd
import numpy as np
import environment2
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from collections import defaultdict

from statsmodels.tsa.stattools import adfuller
import pymannkendall as mk
import warnings;   warnings.filterwarnings("ignore")

In [35]:
env = environment2.environment2()
user_list_2D = env.user_list_2D
user_list_3D = env.user_list_3D

In [36]:
user_list_2D[0]

'data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv'

In [37]:
fileNames2D=['data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv']

In [38]:
fileNames2D=user_list_2D

In [39]:
fileNames2D

array(['data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_8b544d24-3274-4bb0-9719-fd2bccc87b02.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_93be0a02-3f7a-46c9-b526-aa956ed00856.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_954edb7c-

In [40]:
def get_ROISubset(data):
    roi_subset=[]
    subset=1
    for i in range(len(data)):
        if data['State'][i]=='Sensemaking':
            if (i < (len(data) - 1)) and data['State'][i+1]!= 'Sensemaking':
                roi_subset.append(subset)
                subset=subset+1
            else:
                roi_subset.append(subset)
        else:
            roi_subset.append(subset)
    return roi_subset

In [41]:
from collections import defaultdict
def get_probabilities(dataframe):
    dataframe['high-level-action']=np.zeros(len(dataframe))
    probabilities=[]
    map=dict()
    # print(dataframe.head(5))
    states=['Answering','Sensemaking','Foraging','Navigation']
    actions=['change','same']
    for state in states:
      for action in actions:
        map[state+action]=1


    current_state=None
    for i in range(len(dataframe)):
      if dataframe["State"][i]== current_state:
        dataframe["high-level-action"][i]="same"
      else:
        dataframe["high-level-action"][i]="change"
      current_state=dataframe["State"][i]

    for i in range(len(dataframe)):
      if dataframe['State'][i]=="Navigation":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Navigationchange']  + map['Navigationsame'])

      elif dataframe['State'][i]=="Sensemaking":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Sensemakingchange'] + map['Sensemakingsame'])

      elif dataframe['State'][i]=="Foraging":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Foragingchange'] + map['Foragingsame'])
      else:
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Answeringsame'] + map['Answeringchange'])
      probabilities.append(probs)
    dataframe['probabilities']=probabilities
    return dataframe, map

In [42]:
from collections import defaultdict
def get_probabilities_subtask(dataframe):
    dataframe['high-level-action']=np.zeros(len(dataframe))
    probabilities=[]
    map=dict()
    # print(dataframe.head(5))
    states=['Sensemaking','Foraging','Navigation']
    actions=['change','same']
    for state in states:
      for action in actions:
        map[state+action]=1


    current_state=None

    for i in range(len(dataframe)):
      if dataframe['State'][i]=="Navigation":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Navigationchange']  + map['Navigationsame'])

      elif dataframe['State'][i]=="Sensemaking":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Sensemakingchange'] + map['Sensemakingsame'])

      else :
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Foragingchange'] + map['Foragingsame'])

      probabilities.append(probs)
    return probabilities

In [43]:
from collections import defaultdict
def get_probabilities_region(dataframe):
    df=dataframe
    probabilities=[]
    map=defaultdict(lambda: defaultdict(int))

    for i in range(len(df)):
            if df.loc[i, "Most_frequent_region"] in ('NorthernRockiesPlains','Northeast','NorthWest','SouthWest','None'):
                df.loc[i, "Most_frequent_region"]=df.loc[i, "Most_frequent_region"]
            else:
                df.loc[i, "Most_frequent_region"]= 'Other'
            if df.loc[i, "State"] not in ('Sensemaking','Foraging','Navigation'):
                df.loc[i, "State"] = 'Sensemaking'
    # print(dataframe.head(5))
    actions=['Sensemaking','Foraging','Navigation']
    states=['NorthernRockiesPlains','Northeast','NorthWest','SouthWest','None','Other']
    for state in states:
      for action in actions:
        map[state][action]=1

    for i in range(len(df)):
      sum = 0
      map[df['Most_frequent_region'][i]][df['State'][i]] +=1
      for act in map[df['Most_frequent_region'][i]]:
                sum += map[df['Most_frequent_region'][i]][act]
      probs=map[df['Most_frequent_region'][i]][df['State'][i]]   / sum
      probabilities.append(probs)
    df['probabilities']= probabilities
    return df, map

In [44]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
        self.trend= None

    def ADF_Stationarity_Test(self, timeseries, printResults = True):
      #Dickey-Fuller test:
      adfTest = adfuller(timeseries, autolag='AIC')

      self.pValue = adfTest[1]

      if (self.pValue<self.SignificanceLevel):
          self.isStationary = True
      else:
          self.isStationary = False

      if printResults:
          dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
          #Add Critical Values
          for key,value in adfTest[4].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Augmented Dickey-Fuller Test Results:')
          print(dfResults)

    def Mann_Kendall_Stationarity_Test(self, timeseries, printResults = True):
      #Mann Kendall Trend Test:
      mktTest = mk.original_test(timeseries,self.SignificanceLevel)

      self.pValue = mktTest[2]
      self.trend= mktTest[0]
      self.isStationary = mktTest[1]

      if printResults:
          dfResults = pd.Series(mktTest[0:3], index=['Trend Type','Stationarity','P-Value'])
          #Add Critical Values
          for key,value in mktTest[3].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Mann Kendall Trend Test:')
          print(dfResults)

In [45]:
user=[]
ROI_subtask=[]
NorthernRockiesPlainsSensemaking=[]
NorthernRockiesPlainsForaging=[]
NorthernRockiesPlainsNavigation=[]

SouthWestSensemaking=[]
SouthWestForaging=[]
SouthWestNavigation=[]

NorthWestSensemaking=[]
NorthWestForaging=[]
NorthWestNavigation=[]

NortheastSensemaking=[]
NortheastForaging=[]
NortheastNavigation=[]

OtherSensemaking=[]
OtherForaging=[]
OtherNavigation=[]

NoneSensemaking=[]
NoneForaging=[]
NoneNavigation=[]

output_dataframe=pd.DataFrame(columns=['User','ROI_Cycle'
'NorthernRockiesPlainsSensemaking',
'NorthernRockiesPlainsForaging',
'NorthernRockiesPlainsNavigation',

'SouthWestSensemaking',
'SouthWestForaging',
'SouthWestNavigation',

'NortheastSensemaking',
'NortheastForaging',
'NortheastNavigation',


'NorthWestSensemaking',
'NorthWestForaging',
'NorthWestNavigation',


'OtherSensemaking',
'OtherForaging',
'OtherNavigation',


'NoneSensemaking',
'NoneForaging',
'NoneNavigation'])


result_dataframe= pd.DataFrame(columns=['User','ROI_Cycle','NorthernRockiesPlains','SouthWest','NorthWest','Northeast','Other','None'])

for u in fileNames2D:
  print("###########################",u)
  index=[]

  all_dfs=[]
  maps=[]
  df = pd.read_csv(u)
  u=u.lstrip('data/NDSI-2D\\taskname_ndsi-2d-task_')
  #u=u.lstrip('data/NDSI-3D\\taskname_ndsi-3d-task_')
  u=u.rstrip('.csv')

  #df['ROI_Cycle']=get_ROISubset(df)
  df['ROI_Cycle']=get_ROISubset(df)

  # print(df.ROI_Cycle.unique())
  for k in  df['ROI_Cycle'].unique():
    dfs,mappings=get_probabilities_region((df.loc[df['ROI_Cycle'] == k]).reset_index(drop=True))
    mapper=dict()
    mapper['User']=u
    mapper['ROI_Cycle']=k
    print("ROI_Cycle: #######################", k)
    for j in ['NorthernRockiesPlains','SouthWest','NorthWest','Northeast','Other','None']:
        #print("Region: ",j ,"Action Preferred: " ,max(mappings[j], key=mappings[j].get))
        maximum=max(mappings[j], key=mappings[j].get)
        if mappings[j][maximum] > 1 :
            action_preferred=maximum
        else:
            action_preferred='NA'
        mapper[j]=action_preferred
    some_df=pd.DataFrame([mapper])
    result_dataframe = result_dataframe.append(some_df, ignore_index=True)
    all_dfs.append(dfs)
    maps.append(mappings)
    ROI_subtask.append(k)
    user.append(u)

########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv
ROI_Cycle: ####################### 1
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv
ROI_Cycle: ####################### 1
ROI_Cycle: ####################### 2
ROI_Cycle: ####################### 3
ROI_Cycle: ####################### 4
ROI_Cycle: ####################### 5
ROI_Cycle: ####################### 6
ROI_Cycle: ####################### 7
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv
ROI_Cycle: ####################### 1
ROI_Cycle: ####################### 2
ROI_Cycle: ####################### 3
ROI_Cycle: ####################### 4
ROI_Cycle: ####################### 5
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv
ROI_Cycle: ####################### 1
ROI_Cycle: ####################### 2
R

In [48]:

result_dataframe.to_csv("data/NDSI-2D\\region_new-actions_context.csv", index=False)
#output_dataframe.to_csv("data/NDSI-3D\\probability_distribution_roi_cycle_3d.csv", index=False)

In [49]:
result_dataframe.head(20)

,User,ROI_Cycle,NorthernRockiesPlains,SouthWest,NorthWest,Northeast,Other,None
0,userid_3abeecbe-327a-441e-be2a-0dd3763c1d45,1,Sensemaking,NA,NA,NA,Navigation,Foraging
1,userid_44968286-f204-4ad6-a9b5-d95b38e97866,1,NA,NA,Sensemaking,NA,Foraging,Foraging
2,userid_44968286-f204-4ad6-a9b5-d95b38e97866,2,NA,NA,Navigation,NA,NA,Sensemaking
3,userid_44968286-f204-4ad6-a9b5-d95b38e97866,3,Navigation,NA,Navigation,NA,Foraging,Foraging
4,userid_44968286-f204-4ad6-a9b5-d95b38e97866,4,Navigation,NA,NA,Navigation,Foraging,NA
5,userid_44968286-f204-4ad6-a9b5-d95b38e97866,5,Navigation,Navigation,NA,Navigation,Navigation,Navigation
6,userid_44968286-f204-4ad6-a9b5-d95b38e97866,6,Foraging,NA,NA,NA,Sensemaking,Foraging
7,userid_44968286-f204-4ad6-a9b5-d95b38e97866,7,Navigation,NA,NA,NA,Navigation,NA
8,userid_6d49fab8-273b-4a91-948b-ecd14556b049,1,NA,NA,Sensemaking,NA,Foraging,Foraging
9,userid_6d49fab8-273b-4a91-948b-ecd14556b049,2,Navigation,NA,Navigation,Navigation,Foraging,Navigation
